# 🗺️ Planners (with FunctuionCallingStepwise planner)

### Initialize the kernel

In [1]:
#r "nuget: Microsoft.SemanticKernel, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Abstractions, 1.7.1"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha"
#r "nuget: Microsoft.SemanticKernel.Planners.Handlebars, 1.7.1-preview"
#r "nuget: Microsoft.SemanticKernel.Planners.OpenAI, 1.7.1-preview"

#!import Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Plugins;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var kernel = Settings.InitializeKernel();

Installed Packages Microsoft.SemanticKernel, 1.7.1 Microsoft.SemanticKernel.Abstractions, 1.7.1 Microsoft.SemanticKernel.Planners.Handlebars, 1.7.1-preview Microsoft.SemanticKernel.Planners.OpenAI, 1.7.1-preview Microsoft.SemanticKernel.Plugins.Core, 1.7.1-alpha Microsoft.SemanticKernel.Plugins.Web, 1.7.1-alpha

### ⌚️ Let's add a native C# plugin for use by the planner

In [2]:
#!import Helpers/PrintHelper.cs
#!import Plugins/MotorPlugin.cs

kernel.Plugins.AddFromType<MotorPlugin>();

PrintHelper.PrintAllPluginFunctions(kernel);

****** Registered 🔌 Plugins and 📦 Functions ******
🔌 MotorPlugin
   📦 /Backward: Moves the car backward.
🔌 MotorPlugin
   📦 /Forward: Moves the car forward.
🔌 MotorPlugin
   📦 /Stop: Stops the car.
🔌 MotorPlugin
   📦 /TurnLeft: Turns the car anticlockwise.
🔌 MotorPlugin
   📦 /TurnRight: Turns the car clockwise.


### 📋 Prepare kernel arguments

In [3]:
var goal = "You have a tree in front of the car. Avoid it.";
var kernelArguments = new KernelArguments()
{
    ["input"] = goal,
    ["commands"] = "go forward, go backward, turn left, turn right, and stop"
};

### Refine the goal

In [4]:
var commandsPlugin = kernel.CreatePluginFromPromptDirectory(System.IO.Path.Combine(System.IO.Directory.GetCurrentDirectory(), "Plugins", "CommandsPlugin"), "CommandsPlugin");
kernel.Plugins.Add(commandsPlugin);
var semanticFunction = commandsPlugin["ExtractBasicCommands"];
var response = await kernel.InvokeAsync(semanticFunction, kernelArguments);
var refinedGoal = response.GetValue<string>();

kernelArguments["input"] = refinedGoal;

display(refinedGoal)

To navigate the robot car to avoid a tree in front of it, you would execute a sequence of basic motor commands in the following order:

- Go forward (to gain a little momentum if needed)
- Turn left (to steer away from the tree)
- Go forward (to move clear of the tree)
- Turn right (to re-align with the original direction, if necessary)
- Go forward (to put enough distance between the car and the tree)
- Stop (to finish the action of avoiding the tree and return to a stationary state)

### 🗺️🧠 Generate FunctionCallingStepwise Plan

This planner is different from the handlebarsplanner in that it doesn't generate a plan ahead of time, and simply progresses towards its goal.

### 🏃 Let's see it run

In [5]:
using Microsoft.SemanticKernel.Planning;

#pragma warning disable SKEXP0060

var functionCallingStepwisePlannerOptions = new FunctionCallingStepwisePlannerOptions
{
    MaxIterations = 30
};

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");

PrintHelper.PrintAllPluginFunctions(kernel);

🡲
🡵
🡲
🡶
🡲
·


The robot car has successfully performed the sequence to avoid the tree and has now come to a stop.

In [ ]:

var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)

In [ ]:

functionCallingStepwisePlannerOptions.ExcludedPlugins.Add("CommandsPlugin");
var planner = new FunctionCallingStepwisePlanner(functionCallingStepwisePlannerOptions);
var plannerResult = await planner.ExecuteAsync(kernel, refinedGoal);

display(plannerResult.FinalAnswer)